In [ ]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings(action='ignore')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from openai import OpenAI
from transformers import pipeline

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/DACON_DOBAE/data/'
SEED = 42
reset_seeds(SEED)

In [ ]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
with open(f"{DATA_PATH}config.yml") as f:
    CFG = yaml.full_load(f)

# GPT를 활용한 Text Generation

## 1차 : 질문_3-4, 답변_6-10 증강

In [ ]:
def generate_sentences(text):
    client = OpenAI(api_key = CFG["variable"]["OPENAI_API_KEY"])
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': f'''["{text}"] 이 문장을 다른 방식으로 표현해줘.
                                1 문장으로 명확하고 깔끔하게 변경시켜줬으면 좋겠어.'''
            }
        ]
    )
    sentence = response.choices[0].message.content
    return sentence

In [ ]:
column_list = ['질문_1', '질문_2', '답변_1', '답변_2', '답변_3', '답변_4', '답변_5']
for col in tqdm(column_list):
    train[f'{col}_copy'] = train[col].apply(generate_sentences)
train = train.rename(columns={
                    '질문_1_copy : 질문_3',
                    '질문_2_copy : 질문_4',
                    '답변_1_copy : 답변_6',
                    '답변_2_copy : 답변_7',
                    '답변_3_copy : 답변_8',
                    '답변_4_copy : 답변_9',
                    '답변_5_copy : 답변_10'
                     }
)

- 여기까지 완료된 데이터 저장하기

In [ ]:
train.to_csv(f"{DATA_PATH}train_gpt.csv", index=False, encoding='utf-8-sig')

## 2차 : 질문_5-10 추가 증강
- 질문_2-4 -> 질문_5-7
- 질문_5-7 -> 질문_8-10

In [ ]:
train = pd.read_csv(f"{DATA_PATH}train_gpt.csv") # 1차 1:1 증강한 데이터

In [ ]:
# 질문_5-7 증강
column_list = ['질문_2', '질문_3', '질문_4']
for col in tqdm(column_list):
    train[f'{col}_copy'] = train[col].apply(generate_sentences)

In [ ]:
# 질문_8-10 증강
column_list = ['질문_2_copy', '질문_3_copy', '질문_4_copy']
for col in tqdm(column_list):
    train[f'{col}_copy'] = train[col].apply(generate_sentences)

In [ ]:
train = train.rename(columns={
                    '질문_2_copy' : '질문_5',
                    '질문_3_copy' : '질문_6',
                    '질문_4_copy' : '질문_7',
                    '질문_2_copy_copy' : '질문_8',
                    '질문_3_copy_copy' : '질문_9',
                    '질문_4_copy_copy' : '질문_10'
                     }
)

- 여기까지 완료된 데이터 저장하기

In [ ]:
train.to_csv(f"{DATA_PATH}train_gpt_240223.csv", index=False, encoding='utf-8-sig')